<a href="https://colab.research.google.com/github/Cathy-Guang/TB-drug-resistance/blob/main/MultiLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import csv
from sklearn import metrics

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'TB-drug-resistance/' #change this to whatever folder your code is in
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

RESULTS_PATH = os.path.join(GOOGLE_DRIVE_PATH, "results")

Mounted at /content/drive


In [3]:
X_train = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'test_data/X_trainData_1.csv'))
y_train = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'test_data/Y_trainData_1.csv'))
y_train = y_train.replace({-1:0.5})

X_test = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'test_data/X_testData_1.csv'))
y_test = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'test_data/Y_testData_1.csv'))
y_test = y_test.replace({-1:0.5})
y_test

,RIF,INH,PZA,EMB,STR,CIP,CAP,AMK,MOXI,OFLX,KAN
0,0.0,0.0,1.0,0.0,0.5,0.5,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,1.0,0.0,0.5,0.5,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5
996,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5
997,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5
998,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming you have input_data and labels as your input and output data

# Create a Sequential model
model = Sequential()

# Add input layer
model.add(Dense(units=64, input_dim=X_train.shape[1], activation='relu'))

# Add hidden layers
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))

# Add output layer with sigmoid activation for multilabel binary classification
model.add(Dense(units=11, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
90/90 [==============================] - 6s 7ms/step - loss: 0.6175 - accuracy: 0.1076 - val_loss: 0.5431 - val_accuracy: 0.0153
Epoch 2/10
90/90 [==============================] - 0s 4ms/step - loss: 0.5177 - accuracy: 0.1285 - val_loss: 0.5015 - val_accuracy: 0.0250
Epoch 3/10
90/90 [==============================] - 0s 4ms/step - loss: 0.4937 - accuracy: 0.1340 - val_loss: 0.4986 - val_accuracy: 0.0319
Epoch 4/10
90/90 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.1514 - val_loss: 0.4965 - val_accuracy: 0.0430
Epoch 5/10
90/90 [==============================] - 0s 5ms/step - loss: 0.4702 - accuracy: 0.1417 - val_loss: 0.4998 - val_accuracy: 0.1096
Epoch 6/10
90/90 [==============================] - 0s 4ms/step - loss: 0.4623 - accuracy: 0.1833 - val_loss: 0.4995 - val_accuracy: 0.2705
Epoch 7/10
90/90 [==============================] - 0s 6ms/step - loss: 0.4565 - accuracy: 0.2104 - val_loss: 0.4930 - val_accuracy: 0.2497
Epoch 8/10
90/90 [==

In [5]:
def threshold_values(matrix):
    return [[1 if value > 0.7 else 0 if value < 0.3 else 0.5 for value in row] for row in matrix]

In [20]:
drugs = ['RIF','INH','PZA','EMB','STR','CIP','CAP','AMK','MOXI','OFLX','KAN']

In [7]:
def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

    #creating a set of all the unique classes using the actual class list
    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:

        #creating a list of all the classes except the current class
        other_class = [x for x in unique_class if x != per_class]

        #marking the current class as 1 and all other classes as 0
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]

        #using the sklearn metrics method to calculate the roc_auc_score
        roc_auc = metrics.roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc

    return roc_auc_dict

In [27]:
predictions = model.predict(X_test)
ternary_predictions = threshold_values(predictions)
pred_df = pd.DataFrame(ternary_predictions, columns = drugs)

evaluation_result = []
evaluation_result.append(drugs)
rocauc = []
for drug in drugs:
  pred_s = pd.Series(list(pred_df[drug]))
  test_s = pd.Series(list(y_test[drug]))

  condition = (pred_s == 0.5) | (test_s == 0.5)

# Get indices of rows to exclude
  excluded_indices = pred_s[condition].index

# Filter both prediction and test datasets based on the identified indices
  df_pred_filtered = pred_s[~condition]
  # print(df_pred_filtered)
  df_test_filtered = test_s.drop(excluded_indices)

  # print('y',df_test_filtered)
  if df_test_filtered.nunique() == 1 or df_pred_filtered.nunique() == 1 or df_pred_filtered.size == 0:
    cur_rocauc = "NA"
  else:
    cur_rocauc = metrics.roc_auc_score(df_test_filtered, df_pred_filtered)

  print(cur_rocauc)
  rocauc.append(cur_rocauc)

evaluation_result.append(rocauc)
evaluation_result

32/32 [==============================] - 0s 2ms/step
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 906, dtype: float64
y 0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 906, dtype: float64
0.9509876805437553
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 896, dtype: float64
y 0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 896, dtype: float64
0.9513274336283185
0      1.0
2      0.0
5      0.0
6      1.0
9      1.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 692, dtype: float64
y 0      1.0
2      0.0
5      0.0
6      0.0
9      1.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 692, dtype: float64
0.6693295

[['RIF',
  'INH',
  'PZA',
  'EMB',
  'STR',
  'CIP',
  'CAP',
  'AMK',
  'MOXI',
  'OFLX',
  'KAN'],
 [0.9509876805437553,
  0.9513274336283185,
  0.669329573934837,
  0.9255118093609114,
  0.9455544455544456,
  'NA',
  0.8493589743589743,
  0.9947368421052631,
  0.7948717948717949,
  'NA',
  0.9236209335219236]]